เตรียมไลบรารี

In [3]:
import numpy as np 
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import zipfile
import os
import keras
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [4]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))


Num GPUs Available: 2
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


เตรียมข้อมูล

data มาจาก https://www.kaggle.com/datasets/alxmamaev/flowers-recognition

In [5]:
img_size = 224
batch = 64
base_dir = 'flowers/'

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1. / 255,
                                  validation_split=0.2)

train_data = train_datagen.flow_from_directory(base_dir,
                                               target_size=(img_size, img_size),
                                               subset='training',
                                               batch_size=batch,
                                               class_mode='categorical')

test_data = test_datagen.flow_from_directory(base_dir,
                                             target_size=(img_size, img_size),
                                             subset='validation',
                                             batch_size=batch,
                                             class_mode='categorical')


Found 3457 images belonging to 5 classes.
Found 860 images belonging to 5 classes.


สร้างโมเดล CNN

In [6]:
model = Sequential()
model.add(Conv2D(64, (5, 5), padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 28, 28, 64)       0

In [7]:
# ให้ TensorFlow log ว่าใช้ device ไหน
tf.debugging.set_log_device_placement(True)


In [8]:
physical_devices = tf.config.list_physical_devices('GPU')
print("🧠 Found GPU:", physical_devices)

🧠 Found GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


Compile และฝึกโมเดล

In [10]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=30, validation_data=test_data)

Epoch 1/30
55/55 [==============================] - 26s 461ms/step - loss: 1.0742 - accuracy: 0.5728 - val_loss: 0.9775 - val_accuracy: 0.5977
Epoch 2/30
55/55 [==============================] - 29s 528ms/step - loss: 0.9596 - accuracy: 0.6222 - val_loss: 1.1640 - val_accuracy: 0.5244
Epoch 3/30
55/55 [==============================] - 27s 476ms/step - loss: 0.9036 - accuracy: 0.6500 - val_loss: 0.9441 - val_accuracy: 0.6442
Epoch 4/30
55/55 [==============================] - 28s 507ms/step - loss: 0.8377 - accuracy: 0.6766 - val_loss: 0.8596 - val_accuracy: 0.6686
Epoch 5/30
55/55 [==============================] - 26s 469ms/step - loss: 0.7710 - accuracy: 0.7041 - val_loss: 0.8458 - val_accuracy: 0.6698
Epoch 6/30
55/55 [==============================] - 25s 450ms/step - loss: 0.7252 - accuracy: 0.7246 - val_loss: 0.8554 - val_accuracy: 0.6802
Epoch 7/30
55/55 [==============================] - 25s 449ms/step - loss: 0.6774 - accuracy: 0.7399 - val_loss: 0.8200 - val_accuracy: 0.7116

บันทึกโมเดล

In [11]:
model.save('Model.h5')